In [1]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter 1    
#import netgen.gui
from netgen.occ import *
from netgen.csg import *


In [2]:
E, nu = 50, 0.1
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(3)


In [3]:
L = 1
w = 0.2
left = Plane(Pnt(0,0,0), Vec(-1,0,0)).bc("left")
right = Plane(Pnt(L,0,0), Vec(1,0,0)).bc("right")
bottom = Plane(Pnt(0,0,0), Vec(0,-1,0)).bc("bottom")
top = Plane(Pnt(0,w,0), Vec(0,1,0)).bc("top")
front = Plane(Pnt(0,0,0), Vec(0,0,-1)).bc("front")
back = Plane(Pnt(0,0,w), Vec(0,0,1)).bc("back")
cube = left * right * bottom * top * front * back

geo = CSGeometry()
geo.Add(cube)
mesh = Mesh(geo.GenerateMesh(maxh=0.1))
Draw(mesh)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [4]:
fes = VectorH1(mesh, order=3, dirichlet="left")
u,v = fes.TnT()
gfu = GridFunction(fes)

with TaskManager():
    a = BilinearForm(InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(v))).Compile()*dx)
    pre = Preconditioner(a, "bddc")
    a.Assemble()

rho = 0.1
grav = 9.81
force = CoefficientFunction((0,0,-rho*grav))
f = LinearForm(force[2]*v[2]*dx).Assemble()
from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, pre, printrates=True, tol=1e-8)
gfu.vec.data = inv * f.vec

CG iteration 1, residual = 0.16090515907119327     
CG iteration 2, residual = 0.044774883645356964     
CG iteration 3, residual = 0.018416697736701636     
CG iteration 4, residual = 0.007491920762706081     
CG iteration 5, residual = 0.003000954961793781     
CG iteration 6, residual = 0.0008683684730177719     
CG iteration 7, residual = 0.00043069456591761384     
CG iteration 8, residual = 0.00018050729698157838     
CG iteration 9, residual = 7.275622282891673e-05     
CG iteration 10, residual = 2.8489180078394604e-05     
CG iteration 11, residual = 9.860412607602313e-06     
CG iteration 12, residual = 4.228078367987132e-06     
CG iteration 13, residual = 1.8381375679198749e-06     
CG iteration 14, residual = 7.285589439859848e-07     
CG iteration 15, residual = 2.486973201740011e-07     
CG iteration 16, residual = 1.0083278232218271e-07     
CG iteration 17, residual = 3.940230121584218e-08     
CG iteration 18, residual = 1.5572137581635948e-08     
CG iteration 19, re

In [5]:
Draw(gfu, mesh, "Displacement", deformation=True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [6]:
ord_u = 3 
ord_phi = ord_u - 3

# H1-conforming finite element space
fesu = VectorH1(mesh, order=ord_u, dirichlet='left')
fesphi = L2(mesh, order=ord_phi)

fes = FESpace([fesu,fesphi])
# define trial- and test-functions
u, delta = fes.TrialFunction()
v, vphi = fes.TestFunction()
gfu_u = GridFunction(fesu)
gfu_u.Set((0,0,1))

psih = GridFunction(fesphi)
psih.Set(1)

uk = GridFunction(fesu)
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 
phi = CF(-0.1)    

max_PG_it = 100
max_iter_newton = 50

alpha = Parameter(1)
for k in range(1,max_PG_it):

    alpha.Set(2**k)
    print("Iteration of proximal galerkin: ", k)
    # assing uk, psik
    uk.vec.data = gfu_u.vec
    psik.vec.data = psih.vec
    for i in range(max_iter_newton):
                    # the bilinear-form 
        a = BilinearForm(fes)
        rho = 0.1
        grav = 9.81
        force = CoefficientFunction((0,0,-rho*grav))
        with TaskManager():

            # the stress tensor equation
            a += alpha * InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(v))).Compile()*dx
            # proximal galerkin term
            a += delta * v[2] * dx
            a += u[2] * vphi * dx  

            if ord_phi == 0:
                a += -delta * exp(psih) * vphi * dx - eps * (delta * vphi * dx)
            else:
                a+= -delta * exp(psih) * vphi * dx - eps * (grad(delta) * grad(vphi) * dx)
    
            # the right hand side
            lf = LinearForm(fes)
            lf += alpha * force[2] * v[2] * dx 
            lf += (psik - psih) *v [2] * dx
            lf += (phi + exp(psih)) * vphi * dx
        
            a.Assemble()
            lf.Assemble()

        gfu = GridFunction(fes)
        gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * lf.vec

        if Integrate((gfu.components[0]-gfu_u)**2, mesh) < 1e-8:
            print("Newton tol")
            break
        gfu_u.vec.data = gfu.components[0].vec
        psih.vec.data = psih.vec + gfu.components[1].vec
    if Integrate((uk-gfu_u)**2, mesh) < 1e-8:
        print("PG tol")
        Draw(gfu.components[0], mesh, "displacement")
        break
    if k%10 == 0:
        Draw(gfu.components[0], mesh, "displacement")


Iteration of proximal galerkin:  1
Newton tol
Iteration of proximal galerkin:  2
Newton tol
Iteration of proximal galerkin:  3
Newton tol
Iteration of proximal galerkin:  4
Newton tol
Iteration of proximal galerkin:  5
Newton tol
PG tol


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [7]:
import pickle
fil = open("gfu.pkl", "rb")
gfu2 = pickle.load(fil)
Draw(gfu2-gfu.components[0], mesh, "displacement")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [8]:
Draw(gfu2.components[0], mesh, "displacement")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [9]:
import numpy as np
9*(317.8)/19.0228

150.3564144079736